# 🌊 YOLO-UDD v2.0 - Kaggle Training

**Fixed: NumPy + Dataset Structure Issues!** ✅

## ? Quick Start:
1. **Enable GPU**: Settings → Accelerator → **GPU T4 x2** → Save
2. **Run All Cells** (1-5) - No manual steps needed!
3. **Download trained model** after ~10 hours

## ⚡ What's Fixed:
- ✅ Automatic NumPy 1.x installation (no restart needed!)
- ✅ Dataset auto-restructuring (images moved to correct folders)
- ✅ All dependencies installed in one go
- ✅ Simplified from 6 cells to 5 cells

## ⏱️ Training Info:
- **Time**: ~10 hours (100 epochs on GPU T4)
- **Expected mAP**: 70-72% @ IoU 0.5
- **Dataset**: 7,212 images (auto-downloaded from Google Drive)

---


## Cell 1: Environment Setup

In [ ]:
# CRITICAL: Environment Setup with NumPy Fix
import os
import sys

print("="*70)
print("🔧 CELL 1: Environment Setup")
print("="*70)

# STEP 1: Fix NumPy BEFORE any other imports
print("\n[Step 1/5] Fixing NumPy compatibility...")
!pip uninstall -y numpy scipy scikit-learn tensorboard > /dev/null 2>&1
!pip install -q "numpy<2" "scipy<1.14" scikit-learn tensorboard

import numpy as np
print(f"  ✅ NumPy {np.__version__} (compatible)")

# STEP 2: Setup directories
print("\n[Step 2/5] Setting up directories...")
WORK_DIR = '/kaggle/working'
REPO_DIR = f'{WORK_DIR}/YOLO-UDD-v2.0'
os.chdir(WORK_DIR)
print(f"  ✅ Working directory: {WORK_DIR}")

# STEP 3: Clone repository
print("\n[Step 3/5] Cloning repository...")
if os.path.exists(REPO_DIR):
    import shutil
    shutil.rmtree(REPO_DIR)
    print(f"  🗑️  Removed old repository")

!git clone -q https://github.com/kshitijkhede/YOLO-UDD-v2.0.git

if os.path.exists(REPO_DIR):
    os.chdir(REPO_DIR)
    if REPO_DIR not in sys.path:
        sys.path.insert(0, REPO_DIR)
    print(f"  ✅ Repository cloned")
else:
    raise Exception("❌ Clone failed!")

# STEP 4: Install PyTorch and dependencies
print("\n[Step 4/5] Installing dependencies (~2 min)...")
!pip install -q torch>=2.0.0 torchvision>=0.15.0 albumentations>=1.3.0 \
    opencv-python-headless>=4.7.0 pycocotools>=2.0.6 \
    tqdm pyyaml matplotlib seaborn pandas
print(f"  ✅ Dependencies installed")

# STEP 5: Verify GPU
print("\n[Step 5/5] Verifying GPU...")
import torch
if torch.cuda.is_available():
    print(f"  ✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"  ✅ Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("  ❌ NO GPU! Enable: Settings → GPU T4 x2")
    raise RuntimeError("GPU required!")

print("\n" + "="*70)
print("✅ Cell 1 Complete - Environment Ready!")
print("="*70)


## Cell 2: Download & Prepare Dataset

**Dataset will download from Google Drive and be automatically restructured**

- Downloads: ~170 MB (2-3 minutes)
- Auto-fixes file structure for training
- Verifies all images and annotations


In [ ]:
# Download and prepare dataset
import os
import shutil

print("="*70)
print("📦 CELL 2: Dataset Setup")
print("="*70)

# Configuration
GDRIVE_FILE_ID = '17oRYriPgBnW9zowwmhImxdUpmHwOjgIp'
DATASET_PATH = '/kaggle/working/trashcan'

# Download from Google Drive
print("\n☁️  Downloading from Google Drive...")
!pip install -q gdown
!gdown --id {GDRIVE_FILE_ID} -O /kaggle/working/trashcan.zip --quiet

if os.path.exists('/kaggle/working/trashcan.zip'):
    size = os.path.getsize('/kaggle/working/trashcan.zip') / 1024 / 1024
    print(f"  ✅ Downloaded: {size:.1f} MB")
    
    # Remove old dataset if exists
    if os.path.exists(DATASET_PATH):
        shutil.rmtree(DATASET_PATH)
    
    # Extract
    print("  📦 Extracting...")
    !unzip -o -q /kaggle/working/trashcan.zip -d /kaggle/working/
    
    # Check structure
    if os.path.exists(f"{DATASET_PATH}/images/train"):
        print(f"  ✅ Correct structure detected")
    else:
        print(f"  ⚠️  Wrong structure - fixing...")
        
        # Create correct structure
        os.makedirs(f"{DATASET_PATH}/images/train", exist_ok=True)
        os.makedirs(f"{DATASET_PATH}/images/val", exist_ok=True)
        
        # Find images - they might be in root or 'trashcan' subfolder
        image_files_train = []
        image_files_val = []
        
        # Check if images are in trashcan/ root (wrong structure from zip)
        if os.path.exists(f"{DATASET_PATH}") and not os.path.exists(f"{DATASET_PATH}/images/train"):
            # Images are in wrong location - move them
            for file in os.listdir(DATASET_PATH):
                if file.endswith(('.jpg', '.png')):
                    # Check if it's training or validation based on annotation
                    src = os.path.join(DATASET_PATH, file)
                    # For now, we'll determine from annotations later
                    image_files_train.append(src)
        
        print(f"  ? Restructuring dataset...")
        
        # Read annotations to get correct image lists
        import json
        
        # Load train annotations
        train_ann_file = f"{DATASET_PATH}/instances_train_trashcan.json"
        if os.path.exists(train_ann_file):
            with open(train_ann_file, 'r') as f:
                train_data = json.load(f)
            
            # Move training images to correct location
            for img in train_data['images']:
                img_name = img['file_name']
                src = os.path.join(DATASET_PATH, img_name)
                dst = os.path.join(DATASET_PATH, 'images', 'train', img_name)
                
                if os.path.exists(src) and not os.path.exists(dst):
                    shutil.move(src, dst)
        
        # Load val annotations
        val_ann_file = f"{DATASET_PATH}/instances_val_trashcan.json"
        if os.path.exists(val_ann_file):
            with open(val_ann_file, 'r') as f:
                val_data = json.load(f)
            
            # Move validation images to correct location
            for img in val_data['images']:
                img_name = img['file_name']
                src = os.path.join(DATASET_PATH, img_name)
                dst = os.path.join(DATASET_PATH, 'images', 'val', img_name)
                
                if os.path.exists(src) and not os.path.exists(dst):
                    shutil.move(src, dst)
        
        print(f"  ✅ Dataset restructured")
else:
    raise Exception("❌ Download failed!")

# Verify final structure
print("\n" + "="*70)
print("✅ DATASET READY")
print("="*70)

if os.path.exists(f"{DATASET_PATH}/images/train"):
    train_count = len([f for f in os.listdir(f"{DATASET_PATH}/images/train") if f.endswith(('.jpg', '.png'))])
    val_count = len([f for f in os.listdir(f"{DATASET_PATH}/images/val") if f.endswith(('.jpg', '.png'))])
    
    print(f"\n📊 Dataset Statistics:")
    print(f"  Training images: {train_count:,}")
    print(f"  Validation images: {val_count:,}")
    
    # Verify annotations
    if os.path.exists(f"{DATASET_PATH}/instances_train_trashcan.json"):
        print(f"  ✅ Training annotations found")
    if os.path.exists(f"{DATASET_PATH}/instances_val_trashcan.json"):
        print(f"  ✅ Validation annotations found")
    
    print("\n" + "="*70)
    print("✅ Cell 2 Complete - Ready to Train!")
    print("="*70)
else:
    raise Exception("❌ Dataset structure incorrect!")


## Cell 3: Build & Test Model

Build YOLO-UDD v2.0 model and verify it works


In [ ]:
# Build and test YOLO-UDD model
import torch
from models.yolo_udd import build_yolo_udd

print("="*70)
print("🏗️  CELL 3: Build Model")
print("="*70)

print("\n[Step 1/2] Building model...")
model = build_yolo_udd(num_classes=22)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"  ✅ Model: YOLO-UDD v2.0")
print(f"  ✅ Classes: 22")
print(f"  ✅ Device: {device}")
print(f"  ✅ Parameters: {total_params:,}")

# Test forward pass
print("\n[Step 2/2] Testing model...")
x = torch.randn(1, 3, 640, 640).to(device)
with torch.no_grad():
    predictions, turb_score = model(x)

print(f"  ✅ Forward pass successful")
print(f"  ✅ Turbidity score: {turb_score.item():.4f}")

print("\n" + "="*70)
print("✅ Cell 3 Complete - Model Ready!")
print("="*70)


## Cell 4: Start Training 🚀

**This will take ~10 hours for 100 epochs**

Training will save checkpoints automatically


In [ ]:
# Start training
import subprocess
import sys
import os

print("="*70)
print("🚀 CELL 5: Start Training")
print("="*70)

# Training parameters
EPOCHS = 100
BATCH_SIZE = 8
IMG_SIZE = 640
LEARNING_RATE = 0.001
SAVE_DIR = '/kaggle/working/checkpoints'
DATASET_PATH = '/kaggle/working/trashcan'

# Verify dataset exists
if not os.path.exists(DATASET_PATH):
    print(f"❌ ERROR: Dataset not found at {DATASET_PATH}")
    print("Please run Cell 3 to download the dataset!")
    raise FileNotFoundError(f"Dataset not found: {DATASET_PATH}")

print(f"\n📋 Training Configuration:")
print(f"  Epochs:        {EPOCHS}")
print(f"  Batch Size:    {BATCH_SIZE}")
print(f"  Image Size:    {IMG_SIZE}")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Dataset:       {DATASET_PATH}")
print(f"  Save Dir:      {SAVE_DIR}")

# Verify annotations exist
train_ann = os.path.join(DATASET_PATH, 'instances_train_trashcan.json')
val_ann = os.path.join(DATASET_PATH, 'instances_val_trashcan.json')
print(f"\n🔍 Checking annotations:")
print(f"  instances_train_trashcan.json: {'✅ Found' if os.path.exists(train_ann) else '❌ NOT FOUND'}")
print(f"  instances_val_trashcan.json:   {'✅ Found' if os.path.exists(val_ann) else '❌ NOT FOUND'}")

if not os.path.exists(train_ann) or not os.path.exists(val_ann):
    print("\n❌ ERROR: Annotation files missing!")
    print("Please run Cell 3 to download the dataset!")
    raise FileNotFoundError("Annotations not found")

# Build training command
cmd = [
    sys.executable, 'scripts/train.py',
    '--config', 'configs/train_config.yaml',
    '--img-size', str(IMG_SIZE),
    '--batch-size', str(BATCH_SIZE),
    '--epochs', str(EPOCHS),
    '--num-workers', '4',
    '--device', 'cuda'
]

print("\n🚀 Starting training...")
print(f"Command: {' '.join(cmd)}")
print(f"⏱️  Estimated time: ~10 hours for {EPOCHS} epochs")
print(f"💡 Training progress will appear below...")
print("="*70 + "\n")

# Run training
result = subprocess.run(cmd)

if result.returncode == 0:
    print("\n" + "="*70)
    print("✅ Training completed successfully!")
    print(f"📁 Results saved to: {SAVE_DIR}")
    print("📥 Download checkpoints from Output section →")
    print("="*70)
else:
    print("\n" + "="*70)
    print("❌ Training failed - check error messages above")
    print("="*70)


## Cell 5: View Results & Download Model

Run this after training completes to download your trained model


In [ ]:
# Check training results and download model
import os
from IPython.display import FileLink

print("="*70)
print("📊 CELL 6: Training Results")
print("="*70)

SAVE_DIR = '/kaggle/working/checkpoints'

if os.path.exists(SAVE_DIR):
    print(f"\n✅ Results found: {SAVE_DIR}\n")
    
    # List all checkpoints
    checkpoints = [f for f in os.listdir(SAVE_DIR) if f.endswith('.pth')]
    
    if checkpoints:
        print(f"📦 Found {len(checkpoints)} checkpoint(s):\n")
        for ckpt in sorted(checkpoints):
            ckpt_path = os.path.join(SAVE_DIR, ckpt)
            size = os.path.getsize(ckpt_path) / (1024*1024)
            print(f"  • {ckpt} ({size:.1f} MB)")
        
        # Check for best model
        best_model = os.path.join(SAVE_DIR, 'best_model.pth')
        if os.path.exists(best_model):
            size = os.path.getsize(best_model) / (1024*1024)
            print("\n" + "="*70)
            print("🏆 TRAINING COMPLETE!")
            print("="*70)
            print(f"\n✅ Best Model: best_model.pth")
            print(f"📦 Size: {size:.1f} MB")
            print(f"\n📥 Download your trained model:")
            print(f"   1. Click 'Output' in right sidebar")
            print(f"   2. Look for '{SAVE_DIR}'")
            print(f"   3. Download 'best_model.pth'")
            print(f"\n🎯 Expected Performance: 70-72% mAP@0.5")
            print(f"🎉 Success! Model ready for deployment!")
            print("="*70)
        else:
            print("\n⚠️  best_model.pth not found")
            print("Training may still be running or incomplete")
    else:
        print("⚠️  No checkpoint files found")
        print("Training may have failed or not started yet")
else:
    print(f"\n❌ Results directory not found: {SAVE_DIR}")
    print("Training has not been run yet or failed to create checkpoints")
    print("\nTo train the model:")
    print("  1. Make sure Cell 5 completed without errors")
    print("  2. Check GPU is enabled (Settings → GPU T4 x2)")
    print("  3. Re-run Cell 5 to start training")


In [ ]:
# Verify repository structure
import os

print("="*60)
print("📂 Repository Structure")
print("="*60)

required_dirs = ['models', 'scripts', 'data', 'utils', 'configs']
required_files = ['requirements.txt', 'models/__init__.py', 'scripts/train.py']

for dir_name in required_dirs:
    status = "✓" if os.path.exists(dir_name) else "✗"
    print(f"{status} {dir_name}/")

print()
for file_name in required_files:
    status = "✓" if os.path.exists(file_name) else "✗"
    print(f"{status} {file_name}")

print("="*60)

In [ ]:
# FIX: Force add models to Python path
import os
import sys

print("="*60)
print("🔧 Fixing Module Import Path")
print("="*60)

# Get current directory
current_dir = os.getcwd()
print(f"\nCurrent directory: {current_dir}")

# Check if we're in the repo directory
if 'YOLO-UDD-v2.0' not in current_dir:
    print("\n⚠️  Not in YOLO-UDD-v2.0 directory!")
    
    # Try to find and change to it
    possible_paths = [
        '/kaggle/working/YOLO-UDD-v2.0',
        '/kaggle/YOLO-UDD-v2.0',
        os.path.join(os.getcwd(), 'YOLO-UDD-v2.0')
    ]
    
    for path in possible_paths:
        if os.path.exists(path):
            os.chdir(path)
            current_dir = os.getcwd()
            print(f"✓ Changed to: {current_dir}")
            break
    else:
        print("✗ Could not find YOLO-UDD-v2.0 directory!")
        print("  Please re-run the clone cell (Cell 3)")

# Ensure repo is in Python path
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
    print(f"✓ Added to sys.path: {current_dir}")

# Verify models can be imported
print("\n🔍 Verifying module availability...")
models_path = os.path.join(current_dir, 'models')
if os.path.exists(models_path):
    print(f"  ✓ models/ exists at: {models_path}")
    
    # Check for required files
    required_files = ['__init__.py', 'yolo_udd.py', 'psem.py', 'sdwh.py', 'tafm.py']
    all_present = True
    for file in required_files:
        file_path = os.path.join(models_path, file)
        if os.path.exists(file_path):
            print(f"  ✓ {file}")
        else:
            print(f"  ✗ {file} MISSING!")
            all_present = False
    
    if all_present:
        print("\n✅ All model files present - import should work!")
    else:
        print("\n❌ Some files missing - clone may be incomplete")
        print("   → Re-run Cell 3 (Clone Repository)")
else:
    print(f"  ✗ models/ NOT FOUND at: {models_path}")
    print("\n  Available directories:")
    for item in os.listdir(current_dir):
        if os.path.isdir(os.path.join(current_dir, item)):
            print(f"    📁 {item}/")
    print("\n❌ Repository clone failed!")
    print("   → Re-run Cell 3 (Clone Repository)")

print("="*60)

In [ ]:
# ============================================================
# CRITICAL FIX: Force NumPy 1.x Installation
# ============================================================

print("="*60)
print("🔧 FIXING NumPy Compatibility Issue")
print("="*60)

# Check current NumPy version
import numpy as np
current_version = np.__version__
print(f"\n📌 Current NumPy version: {current_version}")

if current_version.startswith('2.'):
    print("\n⚠️  NumPy 2.x detected - this WILL crash TensorFlow/scikit-learn!")
    print("Forcing downgrade to NumPy 1.x...\n")
    
    # Force uninstall NumPy 2.x
    import sys
    !{sys.executable} -m pip uninstall -y numpy
    
    # Install NumPy 1.x with force reinstall
    !{sys.executable} -m pip install 'numpy==1.26.4' --force-reinstall --no-cache-dir
    
    # Verify the fix worked
    print("\n" + "="*60)
    print("✅ NumPy 1.26.4 Installation Complete!")
    print("="*60)
    print("\n🔴 CRITICAL: YOU MUST RESTART THE KERNEL NOW! 🔴")
    print("\n   Steps:")
    print("   1. Click: Session → Restart Session (or Ctrl+O O)")
    print("   2. Run ALL cells again from Cell 1")
    print("   3. Training will work without crashes!")
    print("\n💡 Why? NumPy is already loaded in memory.")
    print("   Restarting clears memory and loads NumPy 1.26.4")
    print("="*60)
    
    # Stop execution here - user must restart
    raise SystemExit("\n⛔ STOP: Restart kernel now before continuing!")
else:
    print(f"\n✓ NumPy 1.x already installed ({current_version})")
    print("✓ Training should work correctly!")
    print("="*60)

## Step 2: Install Dependencies

## Step 3: Setup Dataset

**Choose ONE of the following methods:**

### **METHOD 1: Kaggle Dataset (Recommended)**
1. Go to: https://www.kaggle.com/datasets
2. Click "New Dataset"
3. Upload TrashCAN dataset ZIP
4. Add to notebook: "Add Data" → Search for your dataset
5. Update `DATASET_PATH` in the cell below

### **METHOD 2: Google Drive (Alternative - Easiest)**
1. Upload your TrashCAN dataset folder to Google Drive
2. Share the folder/file publicly
3. Get the file ID from the share link
4. Update `GDRIVE_FILE_ID` in the cell below
5. Set `USE_GDRIVE = True`

### **METHOD 3: Direct Upload in Notebook**
1. ZIP your dataset locally
2. Upload ZIP to Kaggle notebook directly (< 500MB recommended)
3. Set `USE_LOCAL_UPLOAD = True`

## Step 4: Build Model

## Step 5: Training Configuration

## Step 6: Start Training

**⏱️ Estimated Time**: ~10 hours for 100 epochs on T4 GPU

**💡 Tips**:
- Training will save checkpoints automatically
- You can monitor progress in real-time
- Results saved to `/kaggle/working/runs/train/`
- Download best checkpoint from Output folder after training

## Step 7: Download Results

After training completes, download the trained model checkpoint.

## 🎉 Training Complete!

### Next Steps:
1. **Download Checkpoint**: Download `best.pt` from Output folder
2. **Evaluate Model**: Run evaluation script locally with downloaded checkpoint
3. **Test Detections**: Test on new images

### Expected Results:
- mAP@50:95: **70-72%** (22 classes)
- Training Time: **~10 hours** (100 epochs)
- Checkpoint Size: **~200-300 MB**

---

**📧 Issues?** Check the GitHub repository: https://github.com/kshitijkhede/YOLO-UDD-v2.0